# 5 Min Binary Options Predictions on FX

# LSTM method (Long-term short-term memory)

## Import required libraries to train LSTM deep nerual network

In [247]:
import matplotlib
import numpy as np
import pandas as pd
import itertools
import sklearn
import keras
#import tensorflow
import time

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout
from matplotlib import pyplot as plt
from sklearn import preprocessing




import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from mpl_finance import candlestick_ohlc
import numpy as np
import urllib
import datetime as dt



print('Numpy version: ' + np.__version__)
print('Pandas version: ' + pd.__version__)
print('Matplotlib version: ' + matplotlib.__version__)
print('Sklearn version: ' + sklearn.__version__)
print('Keras version: ' + keras.__version__)
#print('Tensorflow-gpu version: ' + tensorflow.__version__)

Numpy version: 1.16.2
Pandas version: 0.24.2
Matplotlib version: 3.0.3
Sklearn version: 0.20.3
Keras version: 2.2.4


### RSI function - Relative strength index

In [248]:
def relative_strength(prices, n=14):

    deltas = np.diff(prices)
    seed = deltas[:n+1]
    up = seed[seed >= 0].sum()/n
    down = -seed[seed < 0].sum()/n
    rs = up/down
    rsi = np.zeros_like(prices)
    rsi[:n] = 100. - 100./(1. + rs)

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # cause the diff is 1 shorter

        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta

        up = (up*(n - 1) + upval)/n
        down = (down*(n - 1) + downval)/n

        rs = up/down
        rsi[i] = 100. - 100./(1. + rs)

    return rsi

## Main feature enginering function

In [249]:
def FeatureEnginering(MyRawData):


    MyRawData['Gmt time'] = pd.to_datetime(MyRawData['Gmt time'], format='%d.%m.%Y %H:%M:%S.000', utc=True)
    
    no_of_std = 2

    MyRawData['MA_30MA'] = MyRawData['Close'].rolling(window=30).mean()
    MyRawData['MA_30MA_std'] = MyRawData['Close'].rolling(window=30).std() 
    MyRawData['MA_30MA_BB_high'] = MyRawData['MA_30MA'] + no_of_std * MyRawData['MA_30MA_std']
    MyRawData['MA_30MA_BB_low'] = MyRawData['MA_30MA'] - no_of_std * MyRawData['MA_30MA_std']

    MyRawData['RSI'] =relative_strength(MyRawData['Close'])

    del MyRawData['Volume']
    del MyRawData['Gmt time']

    return MyRawData



# Function to plot candlesticks

In [250]:
def graph_data_ohlc(dataset):
    fig = plt.figure()
    ax1 = plt.subplot2grid((1,1), (0,0))
    closep=dataset[:,[3]]
    highp=dataset[:,[1]]
    lowp=dataset[:,[2]]
    openp=dataset[:,[0]]
    date=range(len(closep))
    MA_30MA = dataset[:,[4]]
    MA_30MA_std = dataset[:,[5]]
    MA_30MA_BB_low = dataset[:,[6]]
    MA_30MA_BB_high = dataset[:,[7]]
    RSI = dataset[:,[8]]
    
    
    x = 0
    y = len(date)
    ohlc = []
    while x < y:
        append_me = date[x], openp[x], highp[x], lowp[x], closep[x]
        ohlc.append(append_me)
        x+=1
    candlestick_ohlc(ax1, ohlc, width=0.4, colorup='#77d879', colordown='#db3f3f')
    for label in ax1.xaxis.get_ticklabels():
        label.set_rotation(45)
    ax1.xaxis.set_major_locator(mticker.MaxNLocator(10))
    ax1.grid(True)
    plt.xlabel('Candle')
    plt.ylabel('Price')
    plt.title('Candlestick sample representation')
    plt.legend()
    
    ax1.plot(date, MA_30MA)
    ax1.plot(date, MA_30MA_BB_low)
    ax1.plot(date, MA_30MA_BB_high)
    
    plt.subplots_adjust(left=0.09, bottom=0.20, right=0.94, top=0.90, wspace=0.2, hspace=0)
    plt.show()

# Load the dataset

In [338]:
df = pd.read_csv(r'D:\Users\Mike\Desktop\FX data min\EURUSD_2010_2018.csv')


In [339]:
data_set = FeatureEnginering(df)

In [340]:
data_set = data_set.values

In [341]:
#min_max_scaler = preprocessing.MinMaxScaler()
#data_set = min_max_scaler.fit_transform(data_set)

In [342]:
data_set

array([[ 1.43283   ,  1.43283   ,  1.43267   , ...,         nan,
                nan, 56.09756098],
       [ 1.43268   ,  1.43285   ,  1.43268   , ...,         nan,
                nan, 56.09756098],
       [ 1.4328    ,  1.4328    ,  1.43268   , ...,         nan,
                nan, 56.09756098],
       ...,
       [ 1.14645   ,  1.14674   ,  1.14639   , ...,  1.14664198,
         1.14591736, 74.71141637],
       [ 1.14671   ,  1.14671   ,  1.14635   , ...,  1.1466532 ,
         1.1459208 , 57.89215126],
       [ 1.14644   ,  1.14644   ,  1.14627   , ...,  1.1466498 ,
         1.14591887, 50.4640514 ]])

In [343]:
def generator(data, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=128, step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           data.shape[-1]))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = data[indices]
            
            if data[rows[j] + delay][3] > data[rows[j]][3]:
                targets[j] = 1
            else:
                targets[j] = 0
        yield samples, targets

In [344]:
lookback = 20
step = 1
delay = 5
batch_size = 50

train_gen = generator(data_set,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=2000000,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)
val_gen = generator(data_set,
                    lookback=lookback,
                    delay=delay,
                    min_index=2000001,
                    max_index=3000000,
                    step=step,
                    batch_size=batch_size)
test_gen = generator(data_set,
                     lookback=lookback,
                     delay=delay,
                     min_index=3000001,
                     max_index=None,
                     step=step,
                     batch_size=batch_size)

# Liczba kroków pobierania danych z obiektu val_gen 
# zapewniających przetworzenie całego walidacyjnego zbioru danych.
val_steps = (3000000 - 2000001 - lookback) // batch_size

# Liczba kroków pobierania danych z obiektu test_gen, 
# przy której przetworzony zostanie cały testowy zbiór danych.
test_steps = (len(data_set) - 3000001 - lookback) // batch_size

In [345]:
def testooos():
    test = next(train_gen)
    graph_data_ohlc(test[0][0])

In [346]:
print(next(train_gen))

(array([[[ 1.31193   ,  1.31197   ,  1.31187   , ...,  1.31268432,
          1.31173835, 40.77183468],
        [ 1.31186   ,  1.31188   ,  1.31186   , ...,  1.31268746,
          1.31170988, 40.77183468],
        [ 1.31187   ,  1.31187   ,  1.31172   , ...,  1.31269757,
          1.3116711 , 37.87863717],
        ...,
        [ 1.31276   ,  1.3133    ,  1.31269   , ...,  1.31321848,
          1.31148285, 65.41718719],
        [ 1.3133    ,  1.31364   ,  1.31317   , ...,  1.31330179,
          1.31144621, 64.67179807],
        [ 1.31327   ,  1.31327   ,  1.313     , ...,  1.31334779,
          1.31142888, 60.46720996]],

       [[ 1.26909   ,  1.26944   ,  1.26906   , ...,  1.27077006,
          1.26741794, 59.14134593],
        [ 1.26934   ,  1.26937   ,  1.26911   , ...,  1.27068034,
          1.267445  , 59.66282886],
        [ 1.26934   ,  1.26934   ,  1.26897   , ...,  1.27056788,
          1.26747278, 52.24276993],
        ...,
        [ 1.27013   ,  1.27013   ,  1.26966   , ..., 

import time
starttime=time.time()
while True:
    testooos()
    time.sleep(1.0 - ((time.time() - starttime) % 1.0))

In [348]:
###### from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.LSTM(500,dropout=0.2,recurrent_dropout=0.2,input_shape=(None, data_set.shape[-1]),activation='sigmoid', inner_activation='hard_sigmoid'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer=RMSprop(), loss='binary_crossentropy')
history = model.fit_generator(train_gen,
                              steps_per_epoch=50,
                              epochs=200,
                              validation_data=val_gen,
                              validation_steps=val_steps)



C:\Users\Mike\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(500, dropout=0.2, recurrent_dropout=0.2, input_shape=(None, 9), activation="sigmoid", recurrent_activation="hard_sigmoid")`
  


Epoch 1/200
50/50 [==============================] - 2002s 40s/step - loss: 0.7297 - val_loss: 0.6925
Epoch 2/200
50/50 [==============================] - 1992s 40s/step - loss: 0.7086 - val_loss: 0.6926
Epoch 3/200
50/50 [==============================] - 2000s 40s/step - loss: 0.7028 - val_loss: 0.7024
Epoch 4/200
50/50 [==============================] - 2031s 41s/step - loss: 0.7025 - val_loss: 0.6945
Epoch 5/200
50/50 [==============================] - 2080s 42s/step - loss: 0.6999 - val_loss: 0.6937
Epoch 6/200
50/50 [==============================] - 2033s 41s/step - loss: 0.7003 - val_loss: 0.6922
Epoch 7/200
50/50 [==============================] - 2029s 41s/step - loss: 0.7003 - val_loss: 0.6926
Epoch 8/200
50/50 [==============================] - 2033s 41s/step - loss: 0.6972 - val_loss: 0.7162
Epoch 9/200
50/50 [==============================] - 2037s 41s/step - loss: 0.7029 - val_loss: 0.6944
Epoch 10/200
50/50 [==============================] - 2035s 41s/step - loss: 0.694

KeyboardInterrupt: 